# 14.SVD分解

In [15]:
from numpy import *
from numpy import linalg as la

## 14.1.利用Python实现 SVD 

In [16]:
U, sigma, VT = la.svd([[1,1],[7,7]])
print(f"U矩阵：\n{U}")
print(f"sigma：\n{sigma}")
print(f"VT矩阵：\n{VT}")

U矩阵：
[[-0.14142136 -0.98994949]
 [-0.98994949  0.14142136]]
sigma：
[1.00000000e+01 2.82797782e-16]
VT矩阵：
[[-0.70710678 -0.70710678]
 [ 0.70710678 -0.70710678]]


重构

In [4]:
U = mat(U)
VT = mat(VT)
recMat = U[:, 0] * sigma[0] * VT[0, :]
print(f"重构矩阵：\n{recMat}")

重构矩阵：
[[1. 1.]
 [7. 7.]]


## 14.2.读取数据

In [5]:
def loadExData():
    """
    读取数据
    """
    return[[0, 0, 0, 2, 2],
           [0, 0, 0, 3, 3],
           [0, 0, 0, 1, 1],
           [1, 1, 1, 0, 0],
           [2, 2, 2, 0, 0],
           [5, 5, 5, 0, 0],
           [1, 1, 1, 0, 0]]

In [17]:
Data = loadExData()
U, sigma, VT = la.svd(Data)
print(f"sigma：\n{sigma}")

sigma：
[9.64365076e+00 5.29150262e+00 6.49628424e-16 1.43063514e-16
 2.79192092e-17]


## 14.3. 基于协同过滤的推荐引擎 

### 14.3.1.相似度计算

In [12]:
def ecludSim(inA,inB):
    """
    计算欧式距离相似度
    欧式距离相似度 = 1 / (1 + 欧式距离)
    参数：
        inA -- 输入数据A，向量
        inB -- 输入数据B，向量
    返回：
        1.0/(1.0 + la.norm(inA - inB)) -- 欧式距离相似度
    """
    return 1.0/(1.0 + la.norm(inA - inB))

def pearsSim(inA,inB):
    """
    计算pearson相似度
    pearson相似度 = 0.5 + 0.5 * 相关系数
    参数：
        inA -- 输入数据A，向量
        inB -- 输入数据B，向量
    返回：
        0.5+0.5*corrcoef(inA, inB, rowvar = 0)[0][1] -- pearson相似度
    """
    if len(inA) < 3 : return 1.0
    return 0.5+0.5*corrcoef(inA, inB, rowvar = 0)[0][1]

def cosSim(inA,inB):
    """
    计算余弦相似度
    余弦相似度 = (A * B) / (||A|| * ||B||)
    参数：
        inA -- 输入数据A，向量
        inB -- 输入数据B，向量
    返回：
        0.5+0.5*(num/denom) -- 余弦相似度
    """
    #向量内积，分子
    num = float(inA.T*inB)
    #向量范数乘积，分母
    denom = la.norm(inA)*la.norm(inB)
    return 0.5+0.5*(num/denom)

In [18]:
myMat = mat(loadExData())
print(f"数据中第0和4列之间的欧式距离相似度 = {ecludSim(myMat[:,0], myMat[:,4])}")
print(f"数据中第0和0列之间的欧式距离相似度 = {ecludSim(myMat[:,0], myMat[:,0])}")
print(f"数据中第0和4列之间的余弦相似度 = {cosSim(myMat[:,0], myMat[:,4])}")
print(f"数据中第0和0列之间的余弦相似度 = {cosSim(myMat[:,0], myMat[:,0])}")
print(f"数据中第0和4列之间的pearson相似度 = {pearsSim(myMat[:,0], myMat[:,4])}")
print(f"数据中第0和0列之间的pearson相似度 = {pearsSim(myMat[:,0], myMat[:,0])}")

数据中第0和4列之间的欧式距离相似度 = 0.12973190755680383
数据中第0和0列之间的欧式距离相似度 = 1.0
数据中第0和4列之间的余弦相似度 = 0.5
数据中第0和0列之间的余弦相似度 = 1.0
数据中第0和4列之间的pearson相似度 = 0.20596538173840329
数据中第0和0列之间的pearson相似度 = 1.0


## 14.4.示例：餐馆菜肴推荐引擎 

### 14.4.1.基于物品相似度的推荐引擎

In [19]:
def standEst(dataMat, user, simMeas, item):
    """
    估计评分的标准方法
    参数：
        dataMat -- 数据矩阵
        user -- 用户
        simMeas -- 相似度计算方法
        item -- 物件
    返回：
        估计评分
    """
    #物件数
    n = shape(dataMat)[1]
    #初始化相似度计数器
    simTotal = 0.0; ratSimTotal = 0.0
    #对于每一件物体
    for j in range(n):
        #读取用户评分
        userRating = dataMat[user,j]
        #如果没由评分则跳过
        if userRating == 0: continue
        #如果有评分，计算和要估计的物件的重叠
        overLap = nonzero(logical_and(dataMat[:,item].A>0, \
                                      dataMat[:,j].A>0))[0]
        #如果没有重叠，相似度为0
        if len(overLap) == 0: similarity = 0
        #计算相似度
        else: similarity = simMeas(dataMat[overLap,item], \
                                   dataMat[overLap,j])
        #print('the %d and %d similarity is: %f' % (item, j, similarity))
        #加到总相似度中去
        simTotal += similarity
        #总相似度乘以用户评分，得到总相似度估计
        ratSimTotal += similarity * userRating
    #如果总相似度为0，则返回0
    if simTotal == 0: return 0
    #返回估计评分
    else: return ratSimTotal/simTotal

In [29]:
def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst):
    """
    推荐系统
    参数：
        dataMat -- 数据矩阵
        user -- 用户索引，整数
        N -- 最多的推荐结果，整数，默认为3
        simMeas -- 计算相似度的方法，默认为余弦相似度
        estMethod -- 估计评分的方法，默认为标准方法
    返回：
        N个推荐物件
    """
    #找到没有评分的物件
    unratedItems = nonzero(dataMat[user,:].A==0)[1]
    #如果都评分了，就返回一个提示
    if len(unratedItems) == 0: return('you rated everything')
    #初始化物件分数列表
    itemScores = []
    #对于每个未评分的物件
    for item in unratedItems:
        #估计评分
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        #将评分添加到评分列表中
        itemScores.append((item, estimatedScore))
    #排序后返回前N个作为推荐物件
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]

测试

In [24]:
myMat = mat(loadExData())
#对数据稍作修改
myMat[0,1] = myMat[0,0] = myMat[1,0] = myMat[2,0] = 4
myMat[3,3] = 2

print(f"myMat = \n{myMat}")

myMat = 
[[4 4 0 2 2]
 [4 0 0 3 3]
 [4 0 0 1 1]
 [1 1 1 2 0]
 [2 2 2 0 0]
 [5 5 5 0 0]
 [1 1 1 0 0]]


In [25]:
recommend(myMat, 2)

[(2, 2.5), (1, 2.0243290220056256)]

In [26]:
recommend(myMat, 2, simMeas=ecludSim)

[(2, 3.0), (1, 2.8266504712098603)]

In [27]:
recommend(myMat, 2, simMeas=pearsSim)

[(2, 2.5), (1, 2.0)]

### 14.4.2.利用SVD提高推荐的效果

In [28]:
def loadExData2():
    """
    读取数据
    """
    return[[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
           [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
           [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
           [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
           [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
           [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
           [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
           [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
           [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]]

In [30]:
def svdEst(dataMat, user, simMeas, item):
    """
    基于SVD的评分估计
    参数：
        dataMat -- 数据矩阵
        user -- 用户索引，整数
        simMeas -- 相似度计算方法
        item -- 物件索引，整数
    返回：
        估计评分
    """
    #物件个数
    n = shape(dataMat)[1]
    #初始化相似度计数器
    simTotal = 0.0; ratSimTotal = 0.0
    #SVD分解
    U,Sigma,VT = la.svd(dataMat)
    #将Sigma的前4项转化为对角矩阵
    Sig4 = mat(eye(4)*Sigma[:4])
    #重构矩阵
    xformedItems = dataMat.T * U[:,:4] * Sig4.I
    #对重构矩阵进行评分估计
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0 or j==item: continue
        similarity = simMeas(xformedItems[item,:].T,\
                             xformedItems[j,:].T)
        #print('the %d and %d similarity is: %f' % (item, j, similarity))
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal

测试

In [32]:
myMat = mat(loadExData2())

In [33]:
recommend(myMat, 1, estMethod=svdEst)

[(4, 3.344714938469228), (7, 3.3294020724526967), (9, 3.3281008763900686)]

In [34]:
recommend(myMat, 1, estMethod=svdEst, simMeas=pearsSim)

[(4, 3.346952186702173), (9, 3.33537965732747), (6, 3.3071930278130366)]

## 14.5示例：基于 SVD的图像压缩

In [40]:
def printMat(inMat, thresh=0.8):
    """
    打印矩阵
    参数：
        inMat -- 输入矩阵
        thresh -- 分割阈值，默认为0.8
    """
    #遍历矩阵
    for i in range(32):
        for k in range(32):
            #超过阈值就打印1，反之，打印0
            if float(inMat[i,k]) > thresh:
                print(1, end=''),
            else: print(0, end=''),
        print('')

In [38]:
def imgCompress(numSV=3, thresh=0.8):
    """
    图像压缩
    参数：
        numSV -- 奇异值保留个数，整数，默认为3
        thresh -- 阈值，浮点数0-1，默认为0.8
    """
    #初始化数据列表
    myl = []
    #读取数据
    for line in open('0_5.txt').readlines():
        newRow = []
        for i in range(32):
            newRow.append(int(line[i]))
        myl.append(newRow)
    #转化为矩阵
    myMat = mat(myl)
    print("****original matrix******")
    #打印矩阵
    printMat(myMat, thresh)
    #做SVD分解
    U,Sigma,VT = la.svd(myMat)
    #初始化对角矩阵为全零
    SigRecon = mat(zeros((numSV, numSV)))
    #从向量重构对角矩阵
    for k in range(numSV):
        SigRecon[k,k] = Sigma[k]
    #计算重构矩阵
    reconMat = U[:,:numSV]*SigRecon*VT[:numSV,:]
    print("****reconstructed matrix using %d singular values******" % numSV)
    #打印重构后的矩阵
    printMat(reconMat, thresh)

测试

In [41]:
imgCompress(2)

****original matrix******
00000000000000110000000000000000
00000000000011111100000000000000
00000000000111111110000000000000
00000000001111111111000000000000
00000000111111111111100000000000
00000001111111111111110000000000
00000000111111111111111000000000
00000000111111100001111100000000
00000001111111000001111100000000
00000011111100000000111100000000
00000011111100000000111110000000
00000011111100000000011110000000
00000011111100000000011110000000
00000001111110000000001111000000
00000011111110000000001111000000
00000011111100000000001111000000
00000001111100000000001111000000
00000011111100000000001111000000
00000001111100000000001111000000
00000001111100000000011111000000
00000000111110000000001111100000
00000000111110000000001111100000
00000000111110000000001111100000
00000000111110000000011111000000
00000000111110000000111111000000
00000000111111000001111110000000
00000000011111111111111110000000
00000000001111111111111110000000
00000000001111111111111110000000
00000000000111111